## Code to clean all the articles and remove dupe sentences then save to CSV :) (these are the correct CSVs to use!)

First, import and combine all candidate articles.

In [38]:
from textstat.textstat import textstat
import csv
import pandas
import matplotlib
#matplotlib.style.use('ggplot')
%matplotlib inline
import ast
pandas.options.display.max_colwidth = 1000 #for printing long strings

trump_df = pandas.read_csv('data/all_trump_w_topics.csv')
trump_df['candidate'] = 'trump'
clinton_df = pandas.read_csv('data/all_clinton_w_topics.csv')
clinton_df['candidate'] = 'clinton'
sanders_df = pandas.read_csv('data/all_sanders_w_topics.csv')
sanders_df['candidate'] = 'sanders'
cruz_df = pandas.read_csv('data/all_cruz_w_topics.csv')
cruz_df['candidate'] = 'cruz'

n = len(clinton_df) 
clinton_df.index = xrange(len(trump_df), (len(trump_df) + n))
m = len(sanders_df)
sanders_df.index = xrange(max(clinton_df.index), max(clinton_df.index) + m)
c = len(cruz_df)
cruz_df.index = xrange(max(sanders_df.index), max(sanders_df.index) + c)

all_df = pandas.concat([trump_df,clinton_df, sanders_df, cruz_df]) 

# Tokenize & Dedupe Sentences

In [46]:
# Order preserving uniqifier
def uniqify(seq, idfun=None): 
   # order preserving
   if idfun is None:
       def idfun(x): return x
   seen = {}
   result = []
   for item in seq:
       marker = idfun(item)
       # in old Python versions:
       # if seen.has_key(marker)
       # but in new ones:
       if marker in seen: continue
       seen[marker] = 1
       result.append(item)
   return result

In [139]:
# Regex tokenizer test
import re
l = re.compile("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s").split(test) 
l = uniqify(l)
l = [x.replace("\n",'') for x in l]
#print '\n-----\n'.join(l)

In [148]:
import nltk.data
def dedupe(story):
    if type(story) == str:
        #story = re.compile("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s").split(story) # tokenize 
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        story = tokenizer.tokenize(story)
        story = uniqify(story) #uniqify
        story = [sentence.replace("\n", "") for sentence in story] #remove excess whitespace
        story = "\n\n".join(story) # join that shit back up with newline
        return story
    else:
        return None
    
import nltk.data
def sentences(story):
    if type(story) == str:
        #story = re.compile("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s").split(story) # tokenize 
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        story = tokenizer.tokenize(story)
        story = uniqify(story) #uniqify
        story = [sentence.replace("\n", "") for sentence in story] #remove excess whitespace
        return story
    else:
        return None

In [149]:
all_df['body_deduped'] = all_df['body'].apply(lambda x: dedupe(x))
all_df['sentences'] = all_df['body'].apply(lambda x: sentences(x))

In [150]:
all_df.to_csv('data/all_candidates_deduped.csv')

In [136]:
#helper print fun
def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

In [151]:
i = 233
print all_df['url'][i]
print all_df['body'][i]
print ['*' * 100]
print all_df['body_deduped'][i]

http://www.huffingtonpost.com/2016/02/26/hillary-clinton-donald-trump-foundation-speaking-fees_n_9349644.html
Millions of dollars in donations to two charitable foundations associated with Hillary Clinton and Donald Trump may not meet the legal standard for charitable gifts, four tax law experts tell The Huffington Post.
Clinton and Trump command hundreds of thousands of dollars for their speeches and appearances. But not all of this money goes to them. Significant portions of it are directed to the charitable foundations bearing their names.
To qualify as true charitable gifts, donations made to the Donald J. Trump Foundation or the Bill, Hillary and Chelsea Clinton Foundation must come with no strings attached. As the Internal Revenue Service puts it, they must be made "out of detached and disinterested generosity."
But as HuffPost reported, many of the donors to Trump's private charitable foundation got something in return from the billionaire at the time they gave to his foundation